In [1]:
# Useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi


from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity 

from qiskit.circuit import Parameter


from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

shots = 8192 
max_credits = 3 
#job_exp = execute(qc, backend=backend, shots=shots, max_credits=max_credits)
from qiskit.compiler import transpile

In [19]:
#Simulation
#Misura di tutti i qubit
n=6
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i])
    


#circ.draw(output='mpl')

#print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
#print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[-1].parameters)
#circuits[-1].draw(output='mpl')

#circuits[1].draw()

print('Circuit depth')
print(circ.depth())

job = execute(circ,
            backend=BasicAer.get_backend('qasm_simulator'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

phi_range = phi_range
counts =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
#print(counts)
print('\n')
print('Counts of 00')
print(list(map(lambda c: 1/shots*c.get('000000', 0), counts)))


Circuit depth
14


Counts


Counts of 00
[1.0, 0.05078125, 0.81103515625, 0.3846435546875, 0.3818359375, 0.8150634765625, 0.0504150390625, 1.0, 0.0479736328125, 0.8045654296875, 0.39599609375, 0.388671875, 0.80908203125, 0.0484619140625]
